In [1]:
import torch
from torchvision import models
import torch.nn as nn

# 获取alexNet预训练参数
# alexNet = models.alexnet(pretrained=True)
# torch.save(alexNet.state_dict(),'alexNet_weights.pth')
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
from a0_alexNet import show_features
from a0_alexNet import AlexNet

 
Using cpu device


In [2]:
"""
    使用与训练好的参数先预测一波fashionmnis图像分类的准确率
"""
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

resize = 224
trans = transforms.Compose([
    transforms.Resize(resize),
    transforms.ToTensor(),
])
training_data = datasets.FashionMNIST(root = "../data",train=True,download=False,transform=trans)
test_data = datasets.FashionMNIST(root = "../data",train=False,download=True,transform=trans)

mytest_data = []
for idx,(x,y) in enumerate(test_data):
    mytest_data.append((torch.cat((x,x,x),dim=0),y))

train_dataloader = DataLoader(training_data,batch_size=64,shuffle=True)
test_dataloader = DataLoader(mytest_data,batch_size=64,shuffle=True)

In [3]:
"""
    测定test数据集准确率
    batch input [64,1,224,224]
    batch label [64]

    batch output [64,10]
"""
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct,all = 0,0,0
    with torch.no_grad():
        for idx,(X,y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            all += 64
            # X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            if(all % 640 == 0):
                print(f"all number: {all}   correct number: {correct}")
                print(f"Accuracy: {(100*(correct/all)) :> 0.1f}% , avg loss :{test_loss/(idx+1):>8f}")
                print(f"--------------------------------------------------")
    test_loss /= num_batches
    correct /= size
    print(f"test error :\n Accuracy: {(100*correct) :> 0.1f} , avg loss :{test_loss:>8f}")

# 预训练最后输出维度1000 所以目前fashion数据集只能多加个 linear 1000->10
# 后续换成cifar-10 数据集可以正常运行
alexNet = nn.Sequential(
    AlexNet(input_layer=3,num_classes=1000),
    nn.Linear(1000,10),
).to(device)
test(test_dataloader,alexNet,nn.CrossEntropyLoss())

all number: 640   correct number: 71.0
Accuracy:  11.1% , avg loss :2.302317
--------------------------------------------------
all number: 1280   correct number: 135.0
Accuracy:  10.5% , avg loss :2.302852
--------------------------------------------------
all number: 1920   correct number: 190.0
Accuracy:  9.9% , avg loss :2.303447
--------------------------------------------------
all number: 2560   correct number: 254.0
Accuracy:  9.9% , avg loss :2.303333
--------------------------------------------------
all number: 3200   correct number: 315.0
Accuracy:  9.8% , avg loss :2.303320
--------------------------------------------------
all number: 3840   correct number: 370.0
Accuracy:  9.6% , avg loss :2.303298
--------------------------------------------------
all number: 4480   correct number: 424.0
Accuracy:  9.5% , avg loss :2.303272
--------------------------------------------------
all number: 5120   correct number: 478.0
Accuracy:  9.3% , avg loss :2.303099
-------------------

In [6]:
"""
    尝试分解每层的计算
"""
import time
x = torch.rand(size=(10000,3,224,224))
x = x.to(device)
alexNet2 = AlexNet(input_layer = 3,num_classes = 1000)
print(next(alexNet2.parameters()).device)

id = 1
for layer in alexNet2:
    start_time = int(round(time.time() * 1000))
    x = layer(x)
    end_time = int(round(time.time() * 1000))
    # print(x.device)
    print(id,"-",layer.__class__.__name__,f'-output shape:\t',x.shape,
          f'\t computation time: {(end_time - start_time)/1000 :>.3f} s')
    id += 1

cpu
1 - Conv2d -output shape:	 torch.Size([10000, 64, 55, 55]) 	 computation time: 29.499 s
2 - ReLU -output shape:	 torch.Size([10000, 64, 55, 55]) 	 computation time: 2.025 s
3 - MaxPool2d -output shape:	 torch.Size([10000, 64, 27, 27]) 	 computation time: 3.680 s
4 - Conv2d -output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 40.644 s
5 - ReLU -output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 0.189 s
6 - MaxPool2d -output shape:	 torch.Size([10000, 192, 13, 13]) 	 computation time: 2.673 s
7 - Conv2d -output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 10.213 s
8 - ReLU -output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 0.418 s
9 - Conv2d -output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 11.195 s
10 - ReLU -output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 0.166 s
11 - Conv2d -output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 6.970 s
12 - ReLU -output shape:	 t

In [7]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
)

print(len(model))

3


In [55]:
"""
    尝试分解每层的计算量和参数量
"""
x = torch.rand(size=(10000,3,224,224))
x = x.to(device)
alexNet2 = AlexNet(input_layer = 3,num_classes = 1000)
print(next(alexNet2.parameters()).device)

x = show_features(alexNet2,x)

cpu
------------------------------------------------------------------
1-Conv2d computation time: 30.019000 s
output shape: torch.Size([10000, 64, 55, 55]) 	 transport_num:1936000000 	 transport_size:61952.0M
weight  :  parameters size torch.Size([64, 3, 11, 11]) 	 parameters number 23232
bias  :  parameters size torch.Size([64]) 	 parameters number 64
------------------------------------------------------------------
2-ReLU computation time: 2.145000 s
output shape: torch.Size([10000, 64, 55, 55]) 	 transport_num:1936000000 	 transport_size:61952.0M
------------------------------------------------------------------
3-MaxPool2d computation time: 3.739000 s
output shape: torch.Size([10000, 64, 27, 27]) 	 transport_num:466560000 	 transport_size:14929.92M
------------------------------------------------------------------
4-Conv2d computation time: 29.489000 s
output shape: torch.Size([10000, 192, 27, 27]) 	 transport_num:1399680000 	 transport_size:44789.76M
weight  :  parameters size to